In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]


In [3]:
data_train_path = "Dataset/data_train.csv"
data_test_path = "Dataset/data_target_users_test.csv"

In [4]:
df = pd.read_csv(data_train_path)
df_test = pd.read_csv(data_test_path)

df.columns = ["UserID", "ItemID", "Interaction"]
df_test.columns = ["UserID"]

In [5]:
df

,UserID,ItemID,Interaction
0,1,7,1.0
1,1,15,1.0
2,1,16,1.0
3,1,133,1.0
4,1,161,1.0
...,...,...,...
478725,13024,13605,1.0
478726,13024,13823,1.0
478727,13024,15122,1.0
478728,13024,18185,1.0


In [6]:
metric_to_optimize="MAP"
cutoff_to_optimize=10

In [7]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [8]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [10]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = []
for u in urm_validation:
    evaluator_validation.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2581 (20.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2598 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2621 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2601 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2649 (21.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2614 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2608 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2586 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2593 

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2146 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2177 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2157 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2132 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2149 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2129 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2228 (17.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2183 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2140 

In [12]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

recommenderClass = NMFRecommender



In [13]:
import optuna

# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
storage = optuna.storages.RDBStorage(url=mysql_url)

In [14]:
study = optuna.create_study(study_name="NMF_Hybrid_last", direction="maximize", storage=storage, load_if_exists=True, sampler=optuna.samplers.TPESampler())

[I 2024-01-09 23:15:10,074] A new study created in RDB with name: NMF_Hybrid_last


In [15]:
def objective(trial):

    params = {
        'beta_loss': trial.suggest_categorical('beta_loss',["frobenius", "kullback-leibler"]),
        'num_factors': trial.suggest_int('num_factors', 5, 1000),
        'init_type': trial.suggest_categorical('init_type',["random", "nndsvda"]),
        'l1_ratio': trial.suggest_float('l1_ratio', 0.0, 1.0),

    }

    MAP = 0
    for i in range(k_fold):
        recommender = recommenderClass(urm_train[i])
        recommender.fit(**params)

        result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)

        MAP += result_dict["MAP"].item()
    
    MAP /= k_fold
    return MAP

In [16]:
study.optimize(objective, n_trials=200)

NMFRecommender: URM Detected 508 ( 4.0%) users with no interactions.
NMFRecommender: URM Detected 332 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 2.28 min
EvaluatorHoldout: Processed 10057 (100.0%) in 6.69 sec. Users per second: 1502
NMFRecommender: URM Detected 440 ( 3.5%) users with no interactions.
NMFRecommender: URM Detected 325 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 2.25 min
EvaluatorHoldout: Processed 10040 (100.0%) in 5.96 sec. Users per second: 1683
NMFRecommender: URM Detected 479 ( 3.8%) users with no interactions.
NMFRecommender: URM Detected 345 ( 1.6%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 2.49 min
EvaluatorHoldout: Processed 10017 (100.0%) in 6.74 sec. Users per second: 1486
NMFRecommender: 

[I 2024-01-09 23:42:00,657] Trial 0 finished with value: 0.008485907520637001 and parameters: {'beta_loss': 'kullback-leibler', 'num_factors': 776, 'init_type': 'nndsvda', 'l1_ratio': 0.6210032403364062}. Best is trial 1 with value: 0.013353139369974194.


NMFRecommender: URM Detected 508 ( 4.0%) users with no interactions.
NMFRecommender: URM Detected 332 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 2.93 min
EvaluatorHoldout: Processed 10057 (100.0%) in 6.82 sec. Users per second: 1476
NMFRecommender: URM Detected 440 ( 3.5%) users with no interactions.
NMFRecommender: URM Detected 325 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 3.07 min
EvaluatorHoldout: Processed 10040 (100.0%) in 7.46 sec. Users per second: 1346
NMFRecommender: URM Detected 479 ( 3.8%) users with no interactions.
NMFRecommender: URM Detected 345 ( 1.6%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 3.19 min
EvaluatorHoldout: Processed 10017 (100.0%) in 6.08 sec. Users per second: 1649
NMFRecommender: 

[I 2024-01-10 00:17:00,059] Trial 3 finished with value: 0.007801029680815724 and parameters: {'beta_loss': 'kullback-leibler', 'num_factors': 988, 'init_type': 'nndsvda', 'l1_ratio': 0.0032841616851789768}. Best is trial 1 with value: 0.013353139369974194.


NMFRecommender: URM Detected 508 ( 4.0%) users with no interactions.
NMFRecommender: URM Detected 332 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...


...

NMFRecommender: URM Detected 508 ( 4.0%) users with no interactions.
NMFRecommender: URM Detected 332 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 21.81 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 4.26 sec. Users per second: 2359
NMFRecommender: URM Detected 440 ( 3.5%) users with no interactions.
NMFRecommender: URM Detected 325 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 13.65 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 4.43 sec. Users per second: 2266
NMFRecommender: URM Detected 479 ( 3.8%) users with no interactions.
NMFRecommender: URM Detected 345 ( 1.6%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 20.52 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 4.45 sec. Users per second: 2250
NMFRecommende

[I 2024-01-10 09:22:42,064] Trial 375 finished with value: 0.019067801952925185 and parameters: {'beta_loss': 'frobenius', 'num_factors': 177, 'init_type': 'nndsvda', 'l1_ratio': 0.8347621834464428}. Best is trial 249 with value: 0.01945263291087839.


NMFRecommender: URM Detected 508 ( 4.0%) users with no interactions.
NMFRecommender: URM Detected 332 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 41.91 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 4.73 sec. Users per second: 2124
NMFRecommender: URM Detected 440 ( 3.5%) users with no interactions.
NMFRecommender: URM Detected 325 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 34.46 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 4.69 sec. Users per second: 2139
NMFRecommender: URM Detected 479 ( 3.8%) users with no interactions.
NMFRecommender: URM Detected 345 ( 1.6%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 34.77 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 3.79 sec. Users per second: 2640
NMFRecommende

/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:708: RuntimeWarning: overflow encountered in true_divide
  numerator /= denominator
/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:822: RuntimeWarning: invalid value encountered in multiply
  H *= delta_H
[W 2024-01-10 09:26:09,642] Trial 380 failed with parameters: {'beta_loss': 'frobenius', 'num_factors': 281, 'init_type': 'nndsvda', 'l1_ratio': 0.22710370532142782} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_269108/1259330985.py", line 14, in objective
    recommender.fit(**params)
  File "/home/japo/RecSys-Challenge/Recommenders/MatrixFactorization/NMFRecommender.py", line 70, in fit
    nmf_solver.fit

KeyboardInterrupt: 

In [17]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train[i])
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test

NMFRecommender: URM Detected 508 ( 4.0%) users with no interactions.
NMFRecommender: URM Detected 332 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 18.62 sec
EvaluatorHoldout: Processed 10492 (100.0%) in 4.61 sec. Users per second: 2274
NMFRecommender: URM Detected 440 ( 3.5%) users with no interactions.
NMFRecommender: URM Detected 325 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 26.08 sec
EvaluatorHoldout: Processed 10461 (100.0%) in 4.75 sec. Users per second: 2202
NMFRecommender: URM Detected 479 ( 3.8%) users with no interactions.
NMFRecommender: URM Detected 345 ( 1.6%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 15.17 sec
EvaluatorHoldout: Processed 10481 (100.0%) in 4.84 sec. Users per second: 2166
NMFRecommende

0.023765188581028687

In [18]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train_validation[i])
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test

NMFRecommender: URM Detected 249 ( 2.0%) users with no interactions.
NMFRecommender: URM Detected 113 ( 0.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 26.70 sec
EvaluatorHoldout: Processed 10492 (100.0%) in 3.80 sec. Users per second: 2759
NMFRecommender: URM Detected 191 ( 1.5%) users with no interactions.
NMFRecommender: URM Detected 101 ( 0.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 20.77 sec
EvaluatorHoldout: Processed 10461 (100.0%) in 4.71 sec. Users per second: 2222
NMFRecommender: URM Detected 235 ( 1.9%) users with no interactions.
NMFRecommender: URM Detected 105 ( 0.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 20.12 sec
EvaluatorHoldout: Processed 10481 (100.0%) in 4.64 sec. Users per second: 2260
NMFRecommende

0.03002998571420676